In [ ]:
import pandas as pd
import numpy as np

from sklearn.svm import LinearSVC, SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from utils import (
    precision,
    recall,
    f1_score,
    lwlrap
)

import matplotlib.pyplot as plt

In [32]:
X = pd.read_csv("work/train_noisy_features.csv").drop(columns=["fname"])
Y = pd.read_csv("work/train_noisy_labels.csv").drop(columns=["fname"])

In [33]:
mask = (Y.sum(axis=1) == 1)  # 每行求和，等于1的表示只有一个标签
X = X[mask].reset_index(drop=True)
Y = Y[mask].reset_index(drop=True)
Y = Y.values.argmax(axis=1)

In [34]:
X.shape
lenX = len(X)
train = int(lenX * 0.7)

In [35]:
train_X = X[:train]
val_X = X[train:]
train_Y = Y[:train]
val_Y = Y[train:]

In [36]:
std = StandardScaler()
pca = PCA(n_components=256)
X_train = std.fit_transform(train_X)
X_val = std.transform(val_X)
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)
train_X.shape, X_train.shape

/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: divide by zero encountered in matmul
  Q, _ = normalizer(A @ Q)
/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: overflow encountered in matmul
  Q, _ = normalizer(A @ Q)
/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/utils/extmath.py:335: RuntimeWarning: invalid value encountered in matmul
  Q, _ = normalizer(A @ Q)
/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/utils/extmath.py:336: RuntimeWarning: divide by zero encountered in matmul
  Q, _ = normalizer(A.T @ Q)
/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/utils/extmath.py:336: RuntimeWarning: overflow encountered in matmul
  Q, _ = normalizer(A.T @ Q)
/opt/anaconda3/envs/torch/lib/python3.12/site-packages/sklearn/utils/extmath.py:336: RuntimeWarning: invalid value encountered in matmul
  Q, _ = normalizer(A.T @ Q)
/opt/anaconda3/env

((11596, 2474), (11596, 256))

In [37]:
svc = SVC(kernel="rbf")

In [38]:
svc.fit(X_train, train_Y)

SVC()

In [39]:
y_score = svc.decision_function(X_val)
print(y_score.shape)

(4970, 80)


In [40]:
from utils import lwlrap

classes = svc.classes_     
n_classes = len(classes)


class_to_idx = {label: idx for idx, label in enumerate(classes)}


val_Y_onehot = np.zeros((len(val_Y), n_classes), dtype=int)


for i, label in enumerate(val_Y):
    if label in class_to_idx:
        val_Y_onehot[i, class_to_idx[label]] = 1


lwlrap(val_Y_onehot,y_score)

np.float64(0.3604700696299542)